In [1]:
# %pip install langchain-text-splitters langchain-community langchain-openai langchain-chroma sentence-transformers

^C


<!-- %pip install annlite -->

In [ ]:
# !nvcc -V

In [ ]:
# %pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124

In [ ]:
# %pip install pypdf transformers accelerate huggingface_hub 

In [1]:
import torch; print(torch.__version__);
print(torch.cuda.is_available())


2.7.1+cu118
True


In [ ]:
from langchain_community.vectorstores import AnnLite
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from transformers import pipeline

In [4]:
# 1. Load document
loader = PyPDFLoader("Dataset/Discoveries-and-Origin.pdf")
documents = loader.load()

In [ ]:
# Split
splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
docs = splitter.split_documents(documents)

In [ ]:
# Embed and store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = AnnLite.from_documents(docs, embeddings, persist_directory="./annlite_store")

C:\Users\User\AppData\Local\Temp\ipykernel_5304\2727068387.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# retriever init
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})
globals()["retriever"] = retriever

In [ ]:
# small LLM + RAG helper (flan-t5-small)
hf_pipe = pipeline("text2text-generation",  model="google/flan-t5-small", max_length=256, device=0,)

def smolLLM(prompt: str) -> str:
    out = hf_pipe(prompt, do_sample=False)
    # Transformers returns different shapes across versions
    if isinstance(out, list) and len(out) > 0 and isinstance(out[0], dict):
        return (
            out[0].get("generated_text")
            or out[0].get("summary_text")
            or str(out[0])
        )
    return str(out)

Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (837 > 512). Running this sequence through the model will result in indexing errors


Answer: Ibrahim Effendi


In [ ]:
# Robust document-to-text extractor
def extract_text(item):
    """Works with Document, (doc,score) tuples, dicts, etc."""
    try:
        if hasattr(item, "page_content"):
            return item.page_content
        if isinstance(item, (list, tuple)) and len(item) > 0 and hasattr(item[0], "page_content"):
            return item[0].page_content
        if isinstance(item, dict) and "page_content" in item:
            return item["page_content"]
    except Exception:
        pass
    return str(item)

# RAG answer function
def rag_answer(query: str, k: int = 4) -> str:
    docs = None

    # retriever
    try:
        if "retriever" in globals() and hasattr(retriever, "invoke"):
            docs = retriever.invoke(query)
        elif "retriever" in globals() and hasattr(retriever, "get_relevant_documents"):
            docs = retriever.get_relevant_documents(query)
    except Exception:
        docs = None

    # db.similarity_search
    if docs is None:
        try:
            if hasattr(db, "similarity_search"):
                docs = db.similarity_search(query, k=k)
        except Exception:
            docs = None

    # Build context
    if not docs:
        return "No documents retrieved, cannot answer."

    context_parts = [extract_text(d) for d in docs]
    context = "\n---\n".join(context_parts)

    # Prompt + call smolLLM
    prompt = (
        "Use the following context to answer the question. "
        "If the answer is not in the context, say *I don't know*.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {query}\nAnswer concisely:"
    )
    return smolLLM(prompt)

In [14]:
answer = rag_answer("Who invented the fire engine?")
print("Answer:", answer)

Answer: Ibrahim Effendi


In [15]:
answer = rag_answer("Summarise Indigo section within 500 words")
print("Answer:", answer)

Answer: ii.


In [16]:
answer = rag_answer("Who prohibited use of ribbon-loom and when?")
print('Answer: ', answer)

Answer:  The council of Nuremberg, it is said, prohibited the use of them in 1664, as is mentioned in the Hanau work, which I shall soon have occasion to quote. On the 24th of December, the same year, ribbon-looms were prohibited in the Spanish Netherlands.
